In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM
import torch

/home/dayana/.local/share/virtualenvs/arabic_dialect_mt-CYVo2Mud/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# checkpoint = "CohereForAI/aya-101"
checkpoint = "CohereForAI/aya-23-8B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# aya_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
aya_model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float16)

/home/dayana/.local/share/virtualenvs/arabic_dialect_mt-CYVo2Mud/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/dayana/.local/share/virtualenvs/arabic_dialect_mt-CYVo2Mud/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


In [4]:
def aya_translate(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = aya_model.generate(inputs, max_new_tokens=128)
    return tokenizer.decode(outputs[0])

In [6]:
messages = [{"role": "user", "content": "Whats the capital of france?"}]
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

'<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Whats the capital of france?<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>'

In [22]:
def aya_translate(prompt):
    messages = [{"role": "user", "content": prompt}]
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    outputs = aya_model.generate(
        input_ids, 
        max_new_tokens=100, 
        do_sample=True, 
        temperature=0.1,
    )
    prompt_len = len(input_ids[0])
    return tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True)

In [ ]:
src_text = "إذا كان ده هيريحك"
prompt = f"Egyptian Arabic: {src_text}\nModern Standard Arabic: "
out = aya_translate(prompt) 
out

In [ ]:
import re
re.sub(r'<pad>|</s>', '', out).strip()